In [1]:
import random
import gc
import argparse
import os
import time
import copy
import pandas as pd
import numpy as np
from collections import defaultdict

# Sklearn
from sklearn.model_selection import StratifiedKFold

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# WandB
import wandb

# Custom Import
from utils.loader import get_loaders
from utils.train_utils import test
from configs import filter_config
from utils.models import build_model, load_model
from utils.metric import get_metrics

/opt/conda/envs/bollworm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CFG = filter_config.get_config()

In [3]:
 # Create Folds
train_filter = pd.read_csv("data/preproc/train_filter.csv")
fold =0
val_df = train_filter.query("fold==@fold").reset_index(drop=True)

In [4]:
test_df =  val_df#pd.read_csv("data/preproc/val_df.csv")
test_df


,image_id_worm,bw,fold
0,id_0005ef295aafe6acc63587db.jpg,1,0.0
1,id_00750938fb12fad4fd8fad24.jpg,1,0.0
2,id_009d7e2ef3c4f47b14c6c4c3.jpg,0,0.0
3,id_00e5892f475750f80a9f9ce5.jpg,0,0.0
4,id_0128f4371f1b1a678062cff6.jpg,1,0.0
...,...,...,...
1823,id_ff239c464c474c0936fd5d9c.jpg,0,0.0
1824,id_ff52ef03790a72f1ca7c6837.jpg,1,0.0
1825,id_ff6c6dc955223c93456ae3b9.jpg,1,0.0
1826,id_ffc5b54765267f4aa7f015ca.jpg,1,0.0


In [5]:
test_loader= get_loaders(test_df,phase="val",test=True)

In [6]:
!ls runs/handle_white_neg/dropout0.3_efficientnet_b4_aug++ 

checkpoint.pth.tar  model_best.pth.tar


In [7]:
 build_model()

Model Architecture: efficientnet_b4


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=48, out_features=12, bias=True)
          (expand): Linear(in_features=12, out_features=48, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
    

In [8]:
run_path = "runs/handle_white_neg/datacorrected_efficientnet_b4_aug++"
chk_path = f"{run_path}/model_best.pth.tar"
state_dict  =torch.load(chk_path)["state_dict"]
state_dict[ "classifier.fc.2.weight"] = state_dict["classifier.fc.1.weight"]
state_dict["classifier.fc.2.bias"] = state_dict["classifier.fc.1.bias"]
del state_dict["classifier.fc.1.weight"]
del state_dict["classifier.fc.1.bias"]
model =load_model(state_dict).to(CFG.device)

Model Architecture: efficientnet_b4


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Loaded Model Stored


In [9]:

criterion = nn.BCEWithLogitsLoss()

In [10]:
test_data = test(model, test_loader, device=CFG.device)

Test : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:07<00:00,  3.86it/s]


In [11]:
test_data["preds"]

tensor([[1.],
        [1.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]], device='cuda:0')

In [12]:
test_df["preds"] = test_data["preds"].cpu().detach().numpy()
test_df

,image_id_worm,bw,fold,preds
0,id_0005ef295aafe6acc63587db.jpg,1,0.0,1.0
1,id_00750938fb12fad4fd8fad24.jpg,1,0.0,1.0
2,id_009d7e2ef3c4f47b14c6c4c3.jpg,0,0.0,0.0
3,id_00e5892f475750f80a9f9ce5.jpg,0,0.0,0.0
4,id_0128f4371f1b1a678062cff6.jpg,1,0.0,1.0
...,...,...,...,...
1823,id_ff239c464c474c0936fd5d9c.jpg,0,0.0,0.0
1824,id_ff52ef03790a72f1ca7c6837.jpg,1,0.0,1.0
1825,id_ff6c6dc955223c93456ae3b9.jpg,1,0.0,1.0
1826,id_ffc5b54765267f4aa7f015ca.jpg,1,0.0,1.0


In [13]:
test_df[test_df.preds!=test_df.bw]

,image_id_worm,bw,fold,preds
24,id_03e99321b0ef699d61d998a4.jpg,0,0.0,1.0
60,id_08a9c06cb7b912c6e8b07192.jpg,1,0.0,0.0
92,id_0cdcf54855d2007ceba043f6.jpg,0,0.0,1.0
94,id_0d05024d7f6dcfa40ea2c900.jpg,0,0.0,1.0
115,id_0f70fbfe9040967394c18441.jpg,1,0.0,0.0
...,...,...,...,...
1766,id_f6561725e1d389a73e9e1cdf.jpg,0,0.0,1.0
1771,id_f76f903e93da713c8523f4aa.jpg,0,0.0,1.0
1772,id_f78276ef013b313b7bb59b33.jpg,0,0.0,1.0
1774,id_f7aa8ec2a24ee73582f04287.jpg,0,0.0,1.0


In [14]:
get_metrics(test_df["preds"].tolist(), test_df["bw"].tolist())

(0.937636761487965, 0.9552941176470588)

In [14]:
test_df[test_df.preds!=test_df.bw].to_csv("val_filter_wrong_preds.csv",index=False)

In [13]:
# test_old = pd.read_csv("data/preds/test_filter_preds.csv")
# test_old[test_old.preds==0]

,image_id_worm,preds
4,id_007159c1fa015ba6f394deeb.jpg,0.0
5,id_0087ad7987b3f82bc5b5c7d0.jpg,0.0
7,id_00ba116c0f45a71a7e0e652c.jpg,0.0
16,id_012f50e13a438474cc63b7b7.jpg,0.0
17,id_016185ddae947c538eac2c33.jpg,0.0
...,...,...
2789,id_fec8d705a3291a144fb88ef0.jpg,0.0
2793,id_ff1ab253ae19e92a44acba7f.jpg,0.0
2795,id_ff3b7cb6761f3d632e1ff577.jpg,0.0
2800,id_ffbcb27fa549278f47505515.jpg,0.0


In [14]:
# test_df.to_csv("data/preds/test_droupout0.3_filter_preds.csv",index=False)

In [16]:
!ls data/

preds  preproc	raw
